In [1]:
import numpy as np
text = open('data/이광수_무정.txt', encoding='euc-kr').read()
print('말뭉치 크기:', len(text))

말뭉치 크기: 323659


In [3]:
# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60
# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3
# 추출한 시퀀스를 담을 리스트
sentences = []
# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 107867
고유한 글자: 1679
벡터화...


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

In [13]:
from tensorflow.keras.optimizers import RMSprop
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [14]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) # 전체합을 1로 만들어주기 위해 전체개수로 나눠줌
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [15]:
import random
import sys

random.seed(123)
start_index = random.randint(0, len(text) - maxlen - 1)

# 20 에포크 동안 모델을 훈련합니다
for epoch in range(1, 20):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 200개의 글자를 생성합니다
        for i in range(200):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
843/843 [==============================] - 15s 15ms/step - loss: 3.9326
--- 시드 텍스트: "지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의"
------ 온도: 0.2
지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의 마음이 그 사람이 있는 것이 아니하였다. 그러고 형식의 얼굴을 보았다. 그러고 그 사람이 아니하고 마음을 보았다. 그러고 그 사람이 있는 것이 아니하고 그 사람이 있는 것이 없고, 그 사람이 있는 것이 아니하였다. 그러고 그 사람이 있는 것이 아니하고 형식의 얼굴이 있는 것이 아니하고 그 사람이 있는 것이 아니하고 그 사람이 있는 것이 아니하였다. 그러고
------ 온도: 0.5
지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의 일생의 말이 없는 말하는 형식은 형식의 것이 마음이 아니 하였다. 그러고 그 사람이 되어 있어, 그 불을 보고 그만한 사람의 사람이 있는 것이 말하여 대답을 보았다. 그러고 그러한 일이 아니하고 다만 주인이 없고, 우선은 이 말이 아니하였다. 그러고 얼굴을 알아 있는 것이 다 그 고개를 보고 그 말을 하고 나서 사람이 본다. 그러고 영채의 말을 보았다. 
------ 온도: 1.0
지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의 형식은 언암 바자가 이런 동정물으로 ” 하고 고개를 랑하라며,
“에서 이월향이 파는 음물을 조선과 그고 밖가 짜외히 잘 한았다. 그러고 차모기을 자기의 김작렬이가, 선생의 반대(셔렇)이 아니하였다. 선형은, 그 병인도, 아녀야 이놀라는 생각으로 형식의 옥단에 극냐이 …… 허)의 듯하게 생각들을다가, 저 책란 무일인하며 데지 과연 습니다. 그래서 생각하였다
------ 온도: 1.2
지금껏 

KeyboardInterrupt: 